In [1]:
import pyreadr
import pandas as pd
import os
from RaceID import SCseq
import varID_functions
from scipy.sparse import csr_matrix
import numpy as np
import RaceID_utils
import re

In [2]:
input_data = './input_data/'
rds_files = ['wt.rds', 'EPO.rds', 'tf.rds']
data_object = []
for file in rds_files:
    file_path = os.path.join(input_data, file)
    result = pyreadr.read_r(file_path)
    data_object.append(result[None])

In [3]:
wt = data_object[0]

In [4]:
sN = SCseq(wt)

In [5]:

# Apply the regular expression pattern
pattern = "^(mt|Rp(l|s)|Gm\\d)"
CGenes = [gene_name for idx, gene_name in enumerate(sN.expdata.index) if re.match(pattern, gene_name)]
sN.filterdata(mintotal=1000, CGenes=CGenes)


ValueError: operands could not be broadcast together with shapes (3659,) (182,) 

In [10]:
minexpr = 5
minnumber = 5
mintotal = 1000
FGenes = []
#total transcript counts
counts = sN.expdata.sum()
#filtering cells
f = counts >= mintotal
sN.counts = counts[f]
g = (sN.expdata.loc[:, f].apply(lambda x: x >= minexpr, axis=1).sum(axis=1) >= minnumber)
sN.ndata = sN.expdata.loc[:,f].div(sN.counts[f])
genes = sN.ndata.index[g]
genes = genes[~genes.isin(FGenes)]

In [18]:
sN.ndata.loc[genes, :].values.T.shape

(4742, 3659)

In [19]:
sN.ndata.loc[CGenes, :]

,bBM1,bBM2,bBM3,bBM4,bBM5,bBM6,bBM7,bBM8,bBM9,bBM10,...,bBM5241,bBM5253,bBM5266,bBM5272,bBM5282,bBM5290,bBM5297,bBM5304,bBM5312,bBM5315
Gm10000,0.000000,0.000033,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
Gm10010,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
Gm10012,0.000030,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
Gm10015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
Gm10020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mt.Nd3,0.000562,0.000863,0.000663,0.000910,0.000635,0.000715,0.000407,0.000720,0.000301,0.000591,...,0.000000,0.000997,0.001974,0.001750,0.000000,0.000000,0.000966,0.000000,0.00000,0.000000
mt.Nd4,0.002986,0.004316,0.003126,0.004230,0.001765,0.003616,0.003101,0.003267,0.003260,0.002472,...,0.002752,0.004985,0.004936,0.002625,0.003546,0.000951,0.009662,0.008806,0.00691,0.003984
mt.Nd4l,0.000000,0.000066,0.000000,0.000182,0.000071,0.000042,0.000000,0.000111,0.000050,0.000000,...,0.000000,0.000000,0.000987,0.002625,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
mt.Nd5,0.000739,0.000498,0.000663,0.001046,0.000424,0.000505,0.000508,0.001052,0.000652,0.000645,...,0.000917,0.000000,0.000987,0.000000,0.000000,0.000000,0.000966,0.000000,0.00000,0.000000


In [15]:
k = np.corrcoef(sN.ndata.loc[genes, :].values.T, sN.ndata.loc[CGenes, :].values.T)
k

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3659 and the array at index 1 has size 6890